In [ ]:
import pandas as pd
import numpy as np
import joblib
import xgboost
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
import sklearn 
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
data = pd.read_csv("get_around_pricing_project.csv", index_col=0)
data.head()

In [ ]:
# Basic stats
print("Number of rows : {}".format(data.shape[0]))
print()

print("Display of dataset: ")
display(data.head())
print()

print("Basics statistics: ")
data_desc = data.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*data.isnull().sum()/data.shape[0])

In [ ]:
# Separate target variable Y from features X
print("Separating labels from features...")
target_variable = "rental_price_per_day"

X = data.drop(target_variable, axis = 1)
Y = data.loc[:,target_variable]

print("...Done.")
print()

print('Y : ')
print(Y.head())
print()
print('X :')
print(X.head())

In [ ]:
# Division du jeu de données entre ensemble d'apprentissage et ensemble de test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [ ]:
# Automatically detect names of numeric/categorical columns
numeric_features = []
categorical_features = []
for i,t in X.dtypes.items():
    if ('float' in str(t)) or ('int' in str(t)) :
        numeric_features.append(i)
    else :
        categorical_features.append(i)

print('Found numeric features ', numeric_features)
print('Found categorical features ', categorical_features)

In [ ]:
# Create pipeline for numeric features
numeric_transformer = StandardScaler()

# Create pipeline for categorical features
categorical_transformer = OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)
# drop='first' : permet de supprimer une colonne codée, pour éviter la colinéarité.
# handle_unknown='ignore' : permet d'éviter les erreurs si des catégories inconnues apparaissent sur de nouvelles données à étudier (comme la marque ou une nouvelle couleur).


## Combine the transformers into a single object / Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numeric_transformer, numeric_features)
        ])

In [ ]:
preprocesseur = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features),
                                                ('cat', categorical_transformer, categorical_features)])

In [ ]:
modele = Pipeline(steps=[("Preprocessing", preprocesseur),
                         ("Regressor", LinearRegression())
                        ])

In [ ]:
modele.fit(X_train, Y_train)

train_pred = modele.predict(X_train)
test_pred = modele.predict(X_test)

print("r2 on train set : ", r2_score(Y_train, train_pred))
print("r2 on test set : ", r2_score(Y_test, test_pred))

In [ ]:
model = Pipeline(steps=[
        ("Preprocessing", preprocessor),
        ("Regressor",XGBRegressor(learning_rate= 0.1, max_depth= 7, min_child_weight= 7, n_estimators= 170 ))
    ])

In [ ]:
model.fit(X_train, Y_train)

train_predx = model.predict(X_train)
test_predx = model.predict(X_test)

print("r2 on train set : ", r2_score(Y_train, train_predx))
print("r2 on test set : ", r2_score(Y_test, test_predx))

In [ ]:
# dumping model
from joblib import dump, load
dump(model, 'model.joblib')